Step 1: Import the libraries needed

In [1]:
import pandas as pd
import numpy as np
!conda install -c ulmo beautifulsoup4                   
from bs4 import BeautifulSoup
import requests

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                                                   /failed

UnsatisfiableError: The following specifications were found to be incompatible with each other:

Output in format: Requested package -> Available versions

Package requests conflicts for:
requests
tensorboard -> requests[version='>=2.21.0|>=2.21.0,<3']
bokeh -> requests[version='>=1.2.3']
requests-oauthlib -> requests[version='>=2.0.0']
ibm-wsrt-py37main-main -> requests==2.24.0[build=*]
jupyterlab_server -> requests
plotly -> requests
nltk -> requests

Package m

Step 2: Import the data from the webpage

In [2]:
# To fetch the data from the link
Link ="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

data= requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(data, 'lxml')

Step 3: Form the dataframe

In [3]:
# initiating lists that will be used to create dataframe
postalCodeList = []
boroughList = []
neighborhoodList = []

# looking for the necessary information we need to create our dataframe
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        boroughList.append(cells[1].text.rstrip('\n'))
        neighborhoodList.append(cells[2].text.rstrip('\n'))
        
# creating the dataframe with the columns we want using pandas
df1 = pd.DataFrame({"PostalCode": postalCodeList,
                           "Borough": boroughList,
                           "Neighborhood": neighborhoodList})

df1.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Step 4: Clean the dataframe

In [4]:
# Filter all unassigned neighbourhoods
df1 = df1[(df1.Borough.notnull())]
df1 = df1[(df1.Borough != "Not assigned")]

# Group by PostalCode/Borough
df1 = pd.DataFrame(df1.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(list)).reset_index()
df1['Neighborhood'] = df1['Neighborhood'].apply(lambda x: ', '.join(x))

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df1.loc[ (df1.Neighborhood.isnull() == True) |
               (df1.Neighborhood == "Not assigned")
               , 'Neighborhood'] = df1.Borough

df1.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
df1.shape

(103, 3)

Step 5: Import the latitude and longitude data for the postal codes

In [6]:
#using the Geospatial_Coordinates.csv to get respective coordinates for each postal code
df_coor = pd.read_csv('https://cocl.us/Geospatial_data')
df_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Step 6: Merge the longitude and latitude data with the dataframe created

In [7]:
df2=pd.merge(df1, df_coor, how='left', left_on = 'PostalCode', right_on = 'Postal Code')
df2.drop("Postal Code",axis=1,inplace=True)
df2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Step 7: Get the dataframe with the Borough containing "Toronto" 

In [8]:
df3=df2[df2['Borough'].str.contains('Toronto',regex=False)]
df3.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Step 8: Create a map of the neighbourhoods

In [29]:
!pip install folium
import folium
from IPython.display import Image 
from IPython.core.display import HTML 
from IPython.display import display_html

map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df3['Latitude'],df3['Longitude'],df3['Borough'],df3['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

Step 9: Cluster the neighourhood

In [31]:
from sklearn.cluster import KMeans
k=5
toronto_clustering = df3.drop(['PostalCode','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_

array([4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       3, 3, 1, 1, 1, 2, 2, 2, 1, 0, 1, 1, 0, 0, 0, 0, 2, 4], dtype=int32)

Step 10: Create the map with the clusters formed

In [35]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df3['Latitude'], df3['Longitude'], df3['Neighborhood'], df3['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters